In [1]:
# Standart Library
#####
import os
import sys
import time
import string
import random
import pathlib
#####
# Download Library
#####
import matplotlib.pyplot as plt
import torchvision.transforms as tfs
#####
from skimage import io
from PIL import Image, ImageEnhance

In [2]:
def show_save_progress(state, full_lenght):
    sys.stdout.write('\r')
    if full_lenght > 1:
        part = float(state)/(full_lenght-1)
    else:
        part = 1
    symbols_num = int(30 * part)
    sys.stdout.write("[%-30s] %3.2f%%" % ('='*symbols_num, part*100))
    sys.stdout.flush()

def name_generator(size=6, mask="mask"):
    """[summary]
        Generate random name for file with input mask
        Keyword Arguments:
            size {int} -- Lenght of randomized part (default: {6})
            size {str} -- Lenght of randomized part (default: {"mask"})

        Return:
            filename {str} with path in format: /mask_{slug_id}.jpg 
    """
    chars=string.ascii_letters + string.digits
    slug_id = ''.join(random.SystemRandom().choice(chars) for _ in range(size))
    return "/"+mask+"_{}".format(slug_id)+'.jpg'


class CreateDataset(object):
    """[summary]
        Description
        This class create new image instances from input image
        Apply some transforms from torchvision.transforms
    """
    
#     def __init__(self, color_jitter=None, rotation=None, affine=None):
    def __init__(self, color_jitter=None, affine=None):
        """[summary]
            Keyword Arguments:
                color_jitter {dict} -- Params for ColorJitter Transfomation (default: {None})
                rotation {dict} -- Params for RandomRotation Transfomation (default: {None})
                affine {dict} -- Params for RandomAffine Transfomation (default: {None})
        """
        self.color_jitter = self._check_input_in_init(color_jitter, 'Color Jitter Params')
#         self.rotation = self._check_input_in_init(rotation, 'Rotation Params')
        self.affine = self._check_input_in_init(affine, 'Random Affine Params')
        self.set_params() # Set input params to transformations
    
    def __call__(self, img_path=None, num_of_img=None, apply_method='compose'):
        """[summary]
            Keyword Arguments:
                img_path {PurePath} -- path where image located (default: {None})
                num_of_img {int} -- Number of generated images in output (default: {None})
                apply_method {str} -- Order method for transforms (default: {'compose'})
        """
        self.path = img_path
        self.num_of_img = self._check_input_in_call(num_of_img, 'Number of Images') # Check input data
        
        #TODO support others methods
        if isinstance(apply_method, str):
            if apply_method == 'compose':
                print('Your method is {}'.format(apply_method))
                self.compose() # start transoformation jobs. Compose order.
            elif apply_method == 'random_choice':
                print('Your method is {}'.format(apply_method))
                self.random_choice()
            else:
                raise ValueError("Method {} does`n exist.Please choice compose or random_choice".format(apply_method))
        else:
            raise ValueError("{} must be String".format(apply_method))
    
    def save_in_dir(self, image):
        """[summary]
            Save image in directory
            Keyword Arguments:
                image {PIL Image} - transformed image
        """
        img_path = self.path.parent.joinpath(pathlib.PurePath(name_generator(10, 'image')).name)
        image.save(pathlib.Path(img_path))      
    
    def set_params(self):
        self.__transforms__  = [
            tfs.RandomAffine(**self.affine),
            tfs.ColorJitter(**self.color_jitter),
#             tfs.RandomHorizontalFlip(p=0.5)
#             tfs.RandomRotation(**self.rotation),
        ]
    
    def compose(self):
        trfms = tfs.Compose(
            self.__transforms__
        )
        self.create_and_save(trfms)
            
    def random_choice(self):
        trfms = tfs.RandomChoice(
            self.__transforms__
        )
        self.create_and_save(trfms)
    
    def load_image(self):
        path = pathlib.Path(self.path)
        return Image.open(fp=path)
    
    def load_and_preprocess(self):
        img = self.load_image()
        img = img.resize((224,224), resample=0)
        return img
    
    def create_and_save(self, transform):
        img = self.load_and_preprocess()
        print('Processing...')
        for n in range(0, self.num_of_img):
            new_image = transform(img)
            self.save_in_dir(new_image)
            show_save_progress(n, self.num_of_img)
        print('\nAll jobs successfully complected')

    def _check_input_in_init(self, value, name):
        if value is not None:
            if isinstance(value, dict):
                return value
            else:
                raise TypeError("{} must be a dict".format(name))
        else:
            raise ValueError("{} is None".format(name))
    
    def _check_input_in_call(self, value, name):
        if value is not None:
            if isinstance(value, int):
                return value
            else:
                raise TypeError("{} must be Integer".format(name))
        else:
            raise ValueError("{} is None".format(name))
        
        
        
        

In [3]:
def check_empty(structure):
    if structure:
        return structure
    else:
        structure = None

In [4]:
cj_params = {
    'brightness': 0, # optimal param for brightness is 10 
    'contrast': 0, # optimal param for contrast is 10 
    'saturation': 0, # optimal param for saturation is 20 
    'hue': 0.35 # optimal param for saturation is 0.35
}

r_params = {
    'degrees': 40
}

a_params= {
    'degrees': 100, 
    'translate': (0.1, 0.1), 
    'scale': None, 
    'shear': None, 
    'resample': False, 
    'fillcolor': (255, 255, 255)
}

In [7]:
from itertools import combinations_with_replacement

all_keys = ['brightness', 'contrast', 'saturation', 'hue']

def combine_keys_with_replacement(keys):
    combine_keys = list(combinations_with_replacement(keys, len(keys)))
    sorted_keys = list()

    for key in combine_keys:

        if set(key) in sorted_keys:
            continue
        else:
            sorted_keys.append(set(key))

    return sorted_keys

def create_params_for_color_jitter(all_keys, keys_combinations):
    
    brightness = 10
    contrast = 10
    saturation = 10
    hue = 0
    
    cj_params_list = list()
    
    for item in keys_combinations:
        cj_params = dict()
        
        for key in item:
            if key == 'brightness':
                value = brightness
            elif key == 'contrast':
                value = contrast
            elif key == 'saturation':
                value = saturation
            elif key == 'hue':
                value = hue
            else:
                value = None
            
            cj_params.update({key: value})
            
        cj_params_list.append(cj_params)
    
    return cj_params_list

combine_keys = combine_keys_with_replacement(all_keys)
cj_params_list = create_params_for_color_jitter(all_keys, combine_keys)  
# print(cj_params_list) 
print(len(cj_params_list))

15


In [12]:
def main():
    dataset_path = pathlib.Path('../../icecream_dataset/')
    dirs = [x for x in dataset_path.iterdir() if x.is_dir()]
    methods = ['compose', 'random_choice']
    
    counter = 0
    for d in dirs:

        f = list(d.glob('origin.jpg'))
        f = check_empty(f)
        
        

        if f is not None:
            for cj in cj_params_list:
                for m in methods:
                    ds = CreateDataset(color_jitter=cj, affine=a_params)
                    ds(f[0], num_of_img=40, apply_method=m)
                    counter += 1
        
        counter *= 40
        
    print(counter)

In [13]:
import time

start = time.time()
main()
print("--- %s seconds ---" % (time.time() - start))

Your method is compose
Processing...
[==============================] 100.00%
All jobs successfully complected
Your method is random_choice
Processing...
[==============================] 100.00%
All jobs successfully complected
Your method is compose
Processing...
[==============================] 100.00%
All jobs successfully complected
Your method is random_choice
Processing...
[==============================] 100.00%
All jobs successfully complected
Your method is compose
Processing...
[==============================] 100.00%
All jobs successfully complected
Your method is random_choice
Processing...
[==============================] 100.00%
All jobs successfully complected
Your method is compose
Processing...
[==============================] 100.00%
All jobs successfully complected
Your method is random_choice
Processing...
[==============================] 100.00%
All jobs successfully complected
Your method is compose
Processing...
[==============================] 100.00%
All jobs s